# Импорты

In [26]:
import requests
import pqr
import apimoex
import numpy as np
import pandas as pd
from empyrical import *
import os

 # Получение данных
 # ETF

In [55]:
#Режим торгов
board = 'TQTF'

#Чтение списка тикеров из файла
with open("Data/ETFs", "r") as TICKs:
    TICKs = [line.rstrip() for line in TICKs]

process = 0

Data = []


#Загрузка истории тикеров с биржы
with requests.Session() as session:
    for TICK in TICKs:
        process = process + 1
        data = apimoex.get_board_history(session, TICK, board=board)
        if data == []:
             continue
        df = pd.DataFrame(data)
        df = df[['TRADEDATE','CLOSE']]
        df.rename(columns={'CLOSE': TICK}, inplace=True)

        Data.append(df)



#Объединяемм по дате все датасеты тикеров
ETFs = Data[0]
for i in range(1, len(Data)):
    ETFs = pd.merge(ETFs, Data[i], on='TRADEDATE')
ETFs

,TRADEDATE,AKNX,AMCC,AMDG,AMEM,AMGF,AMGM,AMGR,AMHC,AMHY,...,VTBB,VTBE,VTBF,VTBG,VTBH,VTBI,VTBM,VTBU,VTBX,VTBY
0,2021-11-15,1726.5,110.5,106.5,93.7,111.3,97.8,75.7,107.0,98.0,...,121.10,93.00,101.44,1.1242,79.99,95.89,1.0871,100.62,163.70,92.00
1,2021-11-16,1746.0,112.6,107.4,93.7,112.6,98.8,76.2,108.0,98.5,...,121.20,93.40,101.34,1.1240,80.26,96.36,1.0875,101.68,163.25,92.46
2,2021-11-17,1736.5,110.9,105.6,92.3,112.0,97.4,75.5,107.3,97.2,...,121.06,92.25,100.66,1.1235,79.71,96.04,1.0875,100.48,163.35,91.28
3,2021-11-18,1750.5,110.0,106.5,92.0,112.7,97.3,73.7,108.0,98.3,...,121.02,91.97,100.55,1.1258,80.36,97.14,1.0881,101.20,162.50,92.55
4,2021-11-19,1782.5,111.1,107.3,92.8,114.6,97.4,73.2,108.2,98.7,...,120.82,92.49,98.99,1.1250,80.57,97.49,1.0884,101.72,159.25,92.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2022-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,112.56,NaN,70.71,NaN,NaN,NaN,1.1249,NaN,106.95,NaN
96,2022-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,113.76,NaN,73.04,NaN,NaN,NaN,1.1254,NaN,109.90,NaN
97,2022-04-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,113.94,NaN,73.19,NaN,NaN,NaN,1.1262,NaN,111.25,NaN
98,2022-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,113.32,NaN,69.79,NaN,NaN,NaN,1.1268,NaN,106.35,NaN


# RTSTR


In [56]:
#Вручную из файла добавляем RTSTR (из-за отсутствия возможности загрузки через api)
rtstr = pd.read_csv('Data/RTSTR.csv', delimiter=';')
rtstr = rtstr[['<DATE>', '<CLOSE>']]

#Преобразуем дату к стандарту apimoex
for i in range(len(rtstr['<DATE>'])):
    rtstr['<DATE>'].iloc[i] = str(rtstr['<DATE>'].iloc[i])[:4] + '-' + str(rtstr['<DATE>'].iloc[i])[4:6] + '-' + str(rtstr['<DATE>'].iloc[i])[6:]

#Меняем названия колонок
rtstr.rename(columns={'<DATE>': 'TRADEDATE', '<CLOSE>': 'RTSTR'}, inplace=True)
rtstr

,TRADEDATE,RTSTR
0,2016-11-21,1421.37
1,2016-11-22,1424.11
2,2016-11-23,1424.27
3,2016-11-24,1440.95
4,2016-11-25,1426.45
...,...,...
1327,2022-02-22,2359.26
1328,2022-02-24,1428.82
1329,2022-02-25,1801.99
1330,2022-03-24,1639.86


# ОФЗ 20

In [57]:
#Вручную из файла добавляем RTSTR (из-за отсутствия возможности загрузки через api)
OFZ = pd.read_csv('Data/OFZ20.csv', delimiter=',')
OFZ = OFZ[['DATE', 'CLOSE']]

#Преобразуем дату к стандарту apimoex
for i in range(len(OFZ['DATE'])):
    OFZ['DATE'].iloc[i] = str(OFZ['DATE'].iloc[i])[6:] + '-' + str(OFZ['DATE'].iloc[i])[3:5] + '-' + str(OFZ['DATE'].iloc[i])[0:2]

#Меняем названия колонок
OFZ.rename(columns={'DATE': 'TRADEDATE', 'CLOSE': 'OFZ'}, inplace=True)
OFZ

,TRADEDATE,OFZ
0,2022-02-28,"12,110"
1,2022-02-25,"18,420"
2,2022-02-24,"11,860"
3,2022-02-22,"10,650"
4,2022-02-21,"10,390"
...,...,...
2681,2011-12-06,"8,781"
2682,2011-12-05,"8,781"
2683,2011-12-02,"8,781"
2684,2011-12-01,"8,781"


# ПИФы

In [58]:
pif_names = os.listdir('Data/PIFs')

PIFs_tickers = []
Data = []
for pif in pif_names:
    data = pd.read_csv(f'Data/PIFs/{pif}', delimiter=';')
    if not data.empty:
        df = pd.DataFrame(data)
        df = df[['<DATE>','<CLOSE>']]
        df.rename(columns={'<CLOSE>': data['<TICKER>'][0], '<DATE>': 'TRADEDATE'}, inplace=True)
        Data.append(df)
        PIFs_tickers.append(data['<TICKER>'][0])
PIFs = Data[0]
for i in range(1, len(Data)):
    PIFs = pd.merge(PIFs, Data[i], on='TRADEDATE', how='outer')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe0 in position 10: invalid continuation byte

In [59]:
PIFs

,TRADEDATE,ПИФСбер-КН,ПИФМежотрН,ПИФБКСвинв,ПИФМакНГ,ПИФКонсерв,ПИФФондКап,ПИФКарат,ПИФБКСфнн,ПИФСеред,ПИФСрБорДМ,ПИФРСГ-НЖ,ПИФИнтрЭнр,ПИФПервРен,ПИФМдБета,ПИФРВМ-Лог,ПИФТретЗем,ПИФПалКред,ПИФФМФП,AKNX_return
0,05/03/13,92.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.3,NaN,NaN,NaN,NaN,None
1,06/03/13,90.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,12/03/13,90.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,14/03/13,90.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,19/03/13,90.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10750.0,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1688,25/01/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,260.00,None
1689,26/01/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,263.00,None
1690,01/02/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250.00,None
1691,01/10/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255.77,None


# Объединение с RTSTR

In [60]:
#Объединяемм по дате RTSTR с датасетом всех тикеров
ETFs = pd.merge(ETFs, rtstr, on='TRADEDATE')

In [61]:
ETFs

,TRADEDATE,AKNX,AMCC,AMDG,AMEM,AMGF,AMGM,AMGR,AMHC,AMHY,...,VTBE,VTBF,VTBG,VTBH,VTBI,VTBM,VTBU,VTBX,VTBY,RTSTR
0,2021-11-15,1726.5,110.5,106.5,93.7,111.3,97.8,75.7,107.0,98.0,...,93.00,101.44,1.1242,79.99,95.89,1.0871,100.62,163.70,92.00,3393.08
1,2021-11-16,1746.0,112.6,107.4,93.7,112.6,98.8,76.2,108.0,98.5,...,93.40,101.34,1.1240,80.26,96.36,1.0875,101.68,163.25,92.46,3347.89
2,2021-11-17,1736.5,110.9,105.6,92.3,112.0,97.4,75.5,107.3,97.2,...,92.25,100.66,1.1235,79.71,96.04,1.0875,100.48,163.35,91.28,3414.07
3,2021-11-18,1750.5,110.0,106.5,92.0,112.7,97.3,73.7,108.0,98.3,...,91.97,100.55,1.1258,80.36,97.14,1.0881,101.20,162.50,92.55,3337.25
4,2021-11-19,1782.5,111.1,107.3,92.8,114.6,97.4,73.2,108.2,98.7,...,92.49,98.99,1.1250,80.57,97.49,1.0884,101.72,159.25,92.38,3264.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,2022-02-22,1610.5,88.0,109.2,95.9,105.8,106.0,52.1,109.4,102.4,...,94.90,80.61,1.2390,83.65,98.30,1.1109,100.68,124.30,93.79,2359.26
70,2022-02-24,1401.0,90.9,114.0,97.4,109.0,117.0,53.1,115.6,108.4,...,76.50,79.95,1.2167,62.01,63.48,1.1116,73.80,112.80,82.13,1428.82
71,2022-02-25,1521.5,90.3,111.8,96.8,108.9,109.2,54.8,114.8,105.9,...,93.70,65.00,1.2800,83.10,99.50,1.1118,80.06,97.05,70.00,1801.99
72,2022-03-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1639.86


# Высчитываем returns

In [62]:
#Высчитывает returns для каждого тикера
for TICK in TICKs:
    ETFs[f'{TICK}_return'] = None
    for i in range(1, len(ETFs[TICK])):
        x_prev = ETFs[TICK].iloc[i-1]
        x_now = ETFs[TICK].iloc[i]
        ETFs[f'{TICK}_return'].iloc[i] = (x_prev / x_now) / x_prev

ETFs['RTSTR_return'] = None
for i in range(1, len(ETFs['RTSTR'])):
    x_prev = ETFs['RTSTR'].iloc[i-1]
    x_now = ETFs['RTSTR'].iloc[i]
    ETFs['RTSTR_return'].iloc[i] = (x_prev / x_now) / x_prev

#Убираем строки с пустыми значениями
ETFs = ETFs.dropna()

<ipython-input-62-db0d483aed3c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ETFs[f'{TICK}_return'].iloc[i] = (x_prev / x_now) / x_prev
<ipython-input-62-db0d483aed3c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ETFs[f'{TICK}_return'].iloc[i] = (x_prev / x_now) / x_prev
<ipython-input-62-db0d483aed3c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ETFs[f'{TICK}_return'].iloc[i] = (x_prev / x_now) / x_prev
<ipython-input-62-d

In [63]:
ETFs

,TRADEDATE,AKNX,AMCC,AMDG,AMEM,AMGF,AMGM,AMGR,AMHC,AMHY,...,VTBE_return,VTBF_return,VTBG_return,VTBH_return,VTBI_return,VTBM_return,VTBU_return,VTBX_return,VTBY_return,RTSTR_return
9,2021-11-26,1787.5,106.6,108.5,90.8,114.0,95.3,70.8,110.7,99.8,...,0.010975,0.010662,0.890472,0.012225,0.009938,0.9176,0.009643,0.006651,0.010565,0.000332
10,2021-11-29,1781.5,106.0,106.9,90.0,113.6,92.8,68.1,108.9,99.5,...,0.011071,0.010571,0.905387,0.012247,0.010136,0.917515,0.00979,0.00651,0.010779,0.000322
21,2021-12-14,1709.5,97.4,107.8,89.6,110.5,88.1,64.3,108.6,98.8,...,0.011123,0.011025,0.921999,0.012302,0.01023,0.914913,0.009879,0.006937,0.01086,0.00034
22,2021-12-15,1723.0,97.4,108.4,88.6,110.6,87.0,63.8,110.1,99.4,...,0.011189,0.0109,0.917768,0.012297,0.010171,0.914662,0.009901,0.006868,0.010828,0.000342
30,2021-12-27,1781.5,101.3,109.2,90.2,113.7,91.6,66.3,112.9,100.3,...,0.011078,0.010691,0.906618,0.012217,0.010225,0.912326,0.009938,0.0067,0.010858,0.000326
31,2021-12-28,1794.0,101.0,110.9,90.4,115.1,92.3,66.5,113.7,100.4,...,0.011062,0.010622,0.904568,0.012194,0.010167,0.912159,0.009907,0.006673,0.010819,0.000325
32,2021-12-29,1782.5,100.3,111.2,90.0,115.1,92.5,63.7,114.3,101.0,...,0.011041,0.010611,0.902527,0.012102,0.010184,0.911328,0.009881,0.006676,0.010742,0.000329
40,2022-01-12,1742.5,97.6,111.0,95.0,110.9,93.4,59.4,110.4,100.9,...,0.010502,0.010309,0.884877,0.012079,0.010205,0.908513,0.009796,0.006542,0.010593,0.000324
41,2022-01-13,1767.0,96.0,113.2,96.6,112.5,95.6,57.3,112.5,102.8,...,0.01044,0.01054,0.870322,0.011933,0.010072,0.908183,0.009628,0.006777,0.010381,0.000343
42,2022-01-14,1733.5,94.0,111.3,95.5,110.2,94.8,56.8,110.7,102.3,...,0.010476,0.010715,0.870474,0.011906,0.010098,0.907853,0.009724,0.006925,0.010465,0.00035


# Расчеты коэффциентов

In [64]:
RES = pd.DataFrame(index =['Alpha', 'Beta', 'Sharp', 'Treynor', 'Sortino'])

for TICK in TICKs:
    RES[TICK] = None
RES

<ipython-input-64-9860ac3f427a>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  RES[TICK] = None
<ipython-input-64-9860ac3f427a>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  RES[TICK] = None
<ipython-input-64-9860ac3f427a>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  RES[TICK] = None
<ipytho

,AKNX,AMCC,AMDG,AMEM,AMGF,AMGM,AMGR,AMHC,AMHY,AMIG,...,VTBB,VTBE,VTBF,VTBG,VTBH,VTBI,VTBM,VTBU,VTBX,VTBY
Alpha,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Beta,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Sharp,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Treynor,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Sortino,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [71]:
for TICK in TICKs:
    #prob = pd.Series(index=ETFs['TRADEDATE'], data=ETFs[f'{TICK}_return'])
    prob = ETFs[f'{TICK}_return']
    bench = ETFs['RTSTR']
    #RES[TICK].loc['Sharp'] = sharpe_ratio(prob)
    RES[TICK].loc['Sharp'] = prob.mean() / prob.std()
    #RES[TICK].loc['Sortino'] = sortino_ratio(returns=prob)
    #RES[TICK].loc['Alpha'] = alpha(returns=prob, factor_returns=bench, risk_free=0.01)
    #RES[TICK].loc['Beta'] = beta(returns=prob, factor_returns=bench, risk_free=0.01)

9     0.000559
10    0.000561
21    0.000585
22     0.00058
30    0.000561
31    0.000557
32    0.000561
40    0.000574
41    0.000566
42    0.000577
47    0.000601
48    0.000611
49     0.00061
50    0.000594
63    0.000627
66    0.000633
69    0.000621
Name: AKNX_return, dtype: object
9     0.009381
10    0.009434
21    0.010267
22    0.010267
30    0.009872
31    0.009901
32     0.00997
40    0.010246
41    0.010417
42    0.010638
47    0.010881
48    0.011173
49    0.011013
50    0.010787
63     0.01105
66    0.011211
69    0.011364
Name: AMCC_return, dtype: object
9     0.009217
10    0.009355
21    0.009276
22    0.009225
30    0.009158
31    0.009017
32    0.008993
40    0.009009
41    0.008834
42    0.008985
47    0.009132
48    0.009107
49    0.009107
50    0.008953
63    0.009407
66     0.00939
69    0.009158
Name: AMDG_return, dtype: object
9     0.011013
10    0.011111
21    0.011161
22    0.011287
30    0.011086
31    0.011062
32    0.011111
40    0.010526
41    0.010352
4

In [72]:
RES

,AKNX,AMCC,AMDG,AMEM,AMGF,AMGM,AMGR,AMHC,AMHY,AMIG,...,VTBB,VTBE,VTBF,VTBG,VTBH,VTBI,VTBM,VTBU,VTBX,VTBY
Alpha,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Beta,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Sharp,22.799525,16.949204,56.325587,32.573024,27.631025,19.164601,9.638244,49.716242,45.909098,46.549357,...,91.449644,37.357074,20.989061,24.324741,50.490008,53.210465,169.892546,75.828442,15.9181,52.518829
Treynor,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Sortino,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
